In [1]:
import pandas as pd

In [9]:
df = pd.read_csv("../crawler/data/list_house.csv")
len(df)

39998

In [10]:
def check(df):
    l = []
    columns = df.columns
    for col in columns:
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        sum_null = df[col].isnull().sum()
        l.append([col, dtypes, nunique, sum_null])
    df_check = pd.DataFrame(l)
    df_check.columns = ['Column','Dtypes','Nunique','Sum_null']
    return df_check 
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,39998,0
1,Link,object,39996,0
2,Title,object,39411,79
3,Estate_type,object,10,6941
4,Province,object,6,6941
5,District,object,39,6953
6,Ward,object,379,6953
7,Price,object,2305,1280
8,Square,object,835,1816
9,Post_date,object,703,26


In [12]:
df.dropna(subset=["Price", "Square"], inplace=True)

In [13]:
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,37050,0
1,Link,object,37048,0
2,Title,object,36534,44
3,Estate_type,object,10,6721
4,Province,object,6,6721
5,District,object,39,6733
6,Ward,object,365,6733
7,Price,object,2288,0
8,Square,object,733,0
9,Post_date,object,703,0


In [18]:
df = df[(df['Numb_bedroom'] < 100) & (df['Numb_toilet'] < 100) & (df['Numb_floor'] < 100)]

24756

In [17]:
import numpy as np

def convert_price(price_str):
    price_str = price_str.replace(',', '').strip()
    if 'tỷ' in price_str:
        return float(price_str.replace('tỷ', '').strip()) * 1e9
    elif 'triệu' in price_str:
        return float(price_str.replace('triệu', '').strip()) * 1e6
    else:
        try:
            return float(price_str)
        except ValueError:
            return np.nan

In [20]:
df['Price'] = df['Price'].apply(convert_price)
df = df.dropna(subset=['Price'])
print(df['Price'].head())

1    4.860000e+09
2    5.000000e+09
4    5.800000e+10
6    1.700000e+10
7    3.200000e+09
Name: Price, dtype: float64


In [25]:
df = df[(df['Price'] != 0) & (df['Price'] < 500 * 1e9)]
len(df)

23896

In [26]:
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,23896,0
1,Link,object,23894,0
2,Title,object,23618,20
3,Estate_type,object,9,3966
4,Province,object,3,3966
5,District,object,28,3970
6,Ward,object,246,3970
7,Price,float64,1418,0
8,Square,object,457,0
9,Post_date,object,698,0


In [29]:
df.loc[:, 'Province'] = df['Province'].fillna("Hà Nội")
df.dropna(subset=['Estate_type', 'District', 'Ward'], inplace=True)

In [31]:
df = df[df['Province'] == "Hà Nội"]

In [32]:
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,19911,0
1,Link,object,19909,0
2,Title,object,19666,16
3,Estate_type,object,9,0
4,Province,object,1,0
5,District,object,24,0
6,Ward,object,237,0
7,Price,float64,1295,0
8,Square,object,434,0
9,Post_date,object,697,0


In [33]:
df.loc[:, 'Square'] = df['Square'].str.replace(' m2', '', regex=False)

In [34]:
df['Square'] = pd.to_numeric(df['Square'], errors='coerce')
df = df.dropna(subset=['Square'])
print(df['Square'].dtype)

float64


In [35]:
df = df[(df['Square'] != 0) & (df['Square'] < 100000)]

In [320]:
df['Square'].head()

0    54.0
1    33.0
3    55.0
4    99.0
9    41.0
Name: Square, dtype: float64

In [36]:
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,19910,0
1,Link,object,19908,0
2,Title,object,19665,16
3,Estate_type,object,9,0
4,Province,object,1,0
5,District,object,24,0
6,Ward,object,237,0
7,Price,float64,1295,0
8,Square,float64,433,0
9,Post_date,object,697,0


In [37]:
df.to_csv("data_ETLed.csv")